In [70]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random

In [71]:
df = pd.read_csv('Classification_train.csv')

In [72]:
xtrain = np.array(df.iloc[:26000, 1:])
ytrain = np.array(df.iloc[:26000, 0])
xtest = np.array(df.iloc[26001:, 1:].reset_index(drop=True))
ytest = np.array(df.iloc[26001:, 0].reset_index(drop=True))

In [73]:
xtrain = (xtrain - np.mean(xtrain, axis = 1).reshape(-1, 1))/np.std(xtrain, axis = 1).reshape(-1, 1)
xtest = (xtest - np.mean(xtest, axis = 1).reshape(-1, 1))/np.std(xtest, axis = 1).reshape(-1, 1)

In [74]:
# class MLP:
#     def __init__(self,xin_size=len(xtrain[0]),yout_size=len(np.unique(ytrain)),Hidden_Layers=[10,20]):
#         layers = Hidden_Layers+[yout_size]
#         layers = [xin_size]+layers
#         self.depth = len(layers)
#         self.w = [np.random.rand(layers[i+1], layers[i]) for i in range(self.depth-1)]
#         self.b = [np.random.rand(1, layers[i]) for i in range(1, self.depth)]
#         self.z = []
#         self.activ = []
#         print(self.w[0].shape)
#         print(self.b[0].shape)
    
#     def fn(self, z):
#         return np.max(0, z)
    
    # def fprop(self, xinput: np.ndarray, softmax=True) -> np.ndarray:
    #     z = xinput@self.w[0].T+np.tile(self.b[0],[len(xinput), 1])
    #     output = np.maximum(0, z)
    #     self.activ.append(output)
    #     self.z.append(output)
    #     for i in range(1, self.depth-1):
    #         z = output@self.w[i].T+np.tile(self.b[i],[len(xinput), 1])
    #         self.z.append(z)
    #         output = np.maximum(0, z)
    #         self.activ.append(output)
    #     output = (output-np.mean(output, axis=1).reshape(-1, 1))/np.std(output, axis=1).reshape(-1, 1)
    #     output = (np.exp(output))/np.sum(np.exp(output), axis=1).reshape(-1, 1)
    #     return output
    
#     def bprop(self, xinput: np.ndarray, yinput: np.ndarray, softmax=True):
#         m = len(xinput)
#         x = self.fprop(xinput, softmax)
#         x = x.reshape(len(xinput), -1, 1)
#         y = yinput.reshape(-1, 1, 1)
#         dw = []
#         k = np.bool_(self.z[-1].reshape(m, 1, len(self.z[-1][0]))>0).astype(int)
#         layer_deriv = (y/x+(1-y)/(1-x))*x*(1-x) #shape = (22000, 10, 1)
#         layer_deriv = ((layer_deriv*(np.bool_(self.z[-1].reshape(len(self.z[-1]), len(self.z[-1][0]), 1)>0).astype(int))@(self.activ[-1].reshape(m, 1, -1)))@self.w[-1]).T*(np.bool_(self.z[-2].reshape(len(self.z[-2]), len(self.z[-2][0]), 1)>0).astype(int))
#         # layer_deriv = layer_deriv@(np.bool_(self.z[-2].reshape(len(self.z[-2]), 1, len(self.z[-2][0]))>0).astype(int))
#         # dw.append(layer_deriv*self.w[-1])
#         # for i in range(len(self.w)):
#         #     layer_deriv = layer_deriv@(np.bool_(self.z[-i].reshape(len(self.z[-i]), 1, len(self.z[-i][0]))>0).astype(int))
#         #     # deriv = layer_deriv*self.w[-i]
#         #     dw.append(layer_deriv)
#         # last_layer_deriv = ((y/x+(1-y)/(1-x))*x*(1-x)*(np.bool_(self.activ[-1].reshape(len(self.activ[-1]), len((self.activ[-1])[0]), 1)>0).astype(int))).reshape(len(last_layer_deriv),len(last_layer_deriv[0]),1)
#         # dw3 = last_layer_deriv*self.w[-1]
#         # sec_last_layer_deriv = dw3*(np.bool_(x>0).astype(int))
#         # dw2 = sec_last_layer_deriv@self.w[-2]
#         return layer_deriv.shape

In [75]:
import numpy as np
np.random.seed(213)
class MLP:
    def __init__(self, xin_size=len(xtrain[0]), yout_size=len(np.unique(ytrain)), Hidden_Layers=[20, 20]):
        layers = Hidden_Layers + [yout_size]
        layers = [xin_size] + layers
        self.depth = len(layers)
        # Use He initialization for weights
        self.w = [np.random.normal(0, np.sqrt(2 / (layers[i] + layers[i+1])), 
                                size=(layers[i+1], layers[i])) for i in range(self.depth - 1)]
        self.b = [np.zeros((1, layers[i+1])) for i in range(self.depth - 1)]
        self.z = []
        self.activ = []

    def fn(self, z):
        return np.maximum(0, z)
        #return 1/(1+np.exp(-z))
    
    def fn_prime(self, z):
        return (z > 0).astype(int)
        #return self.fn(z)*(1-self.fn(z))

    def fprop(self, xinput: np.ndarray, softmax=True) -> np.ndarray:
        self.z = []
        self.activ = []
        
        # Forward propagation through hidden layers
        output = xinput
        for i in range(self.depth - 1):
            z = output @ self.w[i].T + self.b[i]
            output = self.fn(z)
            self.z.append(z)
            self.activ.append(output)
        
        # Softmax for final layer
        output = np.exp(z) / np.sum(np.exp(z), axis=1).reshape(-1, 1)
        return output

    def bprop(self, xinput: np.ndarray, yinput: np.ndarray, softmax=True):
        m = len(xinput)
        x = self.fprop(xinput, softmax)
        
        delta = (x - yinput.reshape(-1, 1)) / m  # Softmax + Cross-Entropy Loss Gradient
        
        dw = []
        db = []

        for i in reversed(range(self.depth - 1)):
            dw_i = delta.T @ (self.activ[i-1] if i > 0 else xinput)
            db_i = np.sum(delta, axis=0, keepdims=True)
            dw.insert(0, dw_i)
            db.insert(0, db_i)
            
            if i > 0:
                delta = (delta @ self.w[i]) * self.fn_prime(self.z[i-1])
        
        return dw, db
    
    def fit(self, xtrain, ytrain, lr=0.01, iterations=1000):
        for i in range(iterations):
            dw, db = self.bprop(xtrain, ytrain)
            
            # Update weights and biases
            for j in range(self.depth - 1):
                self.w[j] -= lr * dw[j]
                self.b[j] -= lr * db[j]
            
            if i % 10 == 0:  # Print loss every 10 iterations
                loss = -np.mean(np.sum(ytrain.reshape(-1, 1) * np.log(self.fprop(xtrain)), axis=1))
                print(f'Iteration {i}, Loss: {loss}')
    
    def predict(self, xinput):
        output = self.fprop(xinput)
        return np.argmax(output, axis=1)
    
    def accuracy(self, xinput, yinput):
        y_pred = self.predict(xinput)
        # y_true = np.argmax(yinput, axis=1)
        y_true = yinput
        return np.mean(y_pred == y_true)


In [77]:
# Assuming xtrain and ytrain are your training data and labels respectively
xin_size = len(xtrain[0])
yout_size = len(np.unique(ytrain))
model = MLP(xin_size=xin_size, yout_size=yout_size, Hidden_Layers = [100, 100])

# Fit the model 
model.fit(xtrain, ytrain, lr=0.000006, iterations=2000)
model.w[0].shape

# Check accuracy
print('Training Accuracy:', model.accuracy(xtrain, ytrain))
print('Testing Accuracy:', model.accuracy(xtest, ytest))

Iteration 0, Loss: 120.7670104264011
Iteration 10, Loss: 120.74902505704023
Iteration 20, Loss: 120.73431712743293
Iteration 30, Loss: 120.72282694115918
Iteration 40, Loss: 120.71438625146936
Iteration 50, Loss: 120.708964292332
Iteration 60, Loss: 120.70655683403915
Iteration 70, Loss: 120.70713970274302
Iteration 80, Loss: 120.71080547926647
Iteration 90, Loss: 120.71749963488166


KeyboardInterrupt: 